In [16]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
keras.backend.set_image_data_format("channels_last")
import tensorflow as tf

import numpy as np
import random

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [17]:
try:
    os.chdir("/home/das214/PQuant/mdmm_dev/src")
except:
    pass

for f in os.listdir(os.getcwd()):
    print(f)

pquant
smartpixels


In [18]:
dataset_base_dir = "/depot/cms/users/das214/datasets/dataset_3sr/dataset_3sr_16x16_50x12P5_parquets/contained"
tfrecords_base_dir = os.path.join(dataset_base_dir, "TFR_files", "2t")

dataset_train_dir = os.path.join(dataset_base_dir, "train")
dataset_test_dir = os.path.join(dataset_base_dir, "test")
tfrecords_dir_train = os.path.join(tfrecords_base_dir, "TFR_train")
tfrecords_dir_val   = os.path.join(tfrecords_base_dir, "TFR_val")

train_file_size = len(os.listdir(dataset_train_dir))
val_file_size = len(os.listdir(dataset_test_dir))
print(f"train_file_size: {train_file_size}\nval_file_size: {val_file_size}")

from smartpixels.DG.OptimizedDataGenerator_v2 import OptimizedDataGenerator
from smartpixels.losses.maxNLL import custom_loss
from smartpixels.models.conv2D import CreateModel


train_file_size: 80
val_file_size: 20


In [19]:
val_loader = OptimizedDataGenerator(
    load_from_tfrecords_dir= tfrecords_dir_val,
    shuffle=True,
    seed=SEED,
    quantize=False,
)

train_loader = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_train,
    shuffle=True,
    seed=SEED,
    quantize=False,
)


Loading metadata from /depot/cms/users/das214/datasets/dataset_3sr/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val/metadata.json


Loading metadata from /depot/cms/users/das214/datasets/dataset_3sr/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_train/metadata.json


In [20]:
model=CreateModel(shape = (16,16,2), 
                  n_filters=5, SepConv2D_kernel_size=3, 
                  Conv2D_kernel_size=1,
                  pool_size=3, 
                  hidden=16,
                  output = 14)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 16, 16, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 14, 14, 5)      │            33 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 5)      │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 4, 4, 5)        │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 4, 4, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 14)             │           238 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,869 (7.30 KB)

 Trainable params: 1,869 (7.30 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# model.compile(
#     optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
#     loss=custom_loss,
# )
# history = model.fit(
#         x=train_loader,
#         validation_data=val_loader,
#         epochs=5,
#         shuffle=False,
#         verbose=1
#     )

## Add pruning and quantization
To add pruning and quantization, we need a config file that defines how to do that. Let's load a config file from `pquant/configs/`. <br/>
The training function we use later will add the pruning layers and quantized activations automatically using this config

In [22]:
from pquant import get_default_config
from IPython.display import JSON

# pruning_methods: "autosparse, cl, cs, dst, pdp, wanda, mdmm"
pruning_method = "mdmm"
config = get_default_config(pruning_method)
JSON(config)

<IPython.core.display.JSON object>

In [23]:
# Replace layers with compressed layers
from pquant import add_compression_layers
input_shape = (1, 16, 16, 2)

model = add_compression_layers(model, config, input_shape)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 16, 16, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_separable_con… │ (None, 14, 14, 5)      │            41 │
│ (CompressedLayerSeparableConv2… │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_15 (Activation)      │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_conv2d_keras_3 │ (None, 14, 14, 5)      │            34 │
│ (CompressedLayerConv2dKeras)    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 4, 4, 5)        │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 4, 4, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_dense_keras_3  │ (None, 16)             │         1,300 │
│ (CompressedLayerDenseKeras)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_18 (Activation)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_dense_keras_4  │ (None, 16)             │           276 │
│ (CompressedLayerDenseKeras)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_19 (Activation)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_dense_keras_5  │ (None, 14)             │           242 │
│ (CompressedLayerDenseKeras)     │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,893 (7.39 KB)

 Trainable params: 1,869 (7.30 KB)

 Non-trainable params: 24 (96.00 B)

## Create data set
#### Let's create the data loader and the training and validation loops

In [24]:
from quantizers.fixed_point.fixed_point_ops import get_fixed_quantizer
# Set up input quantizer
quantizer = get_fixed_quantizer(overflow_mode="SAT")

from tqdm import tqdm
from pquant import get_layer_keep_ratio, get_model_losses
import re

def get_lambda_values(model):
    """Extracts lambda values with clean, descriptive keys for the progress bar."""
    lambda_info = {}
    # Use counters for each layer type to ensure unique, short keys
    counters = {'conv2d': 1, 'dense': 1, 'other': 1}

    for layer in model.layers:
        if "CompressedLayer" in layer.__class__.__name__ and hasattr(layer, 'pruning_layer'):
            pruning_layer = layer.pruning_layer
            if hasattr(pruning_layer, 'constraint_layer') and hasattr(pruning_layer.constraint_layer, 'lmbda'):
                layer_name = layer.name.lower()
                
                # Assign a clean, short key based on the layer type
                if 'conv2d' in layer_name:
                    key = f"λc2D{counters['conv2d']}"
                    counters['conv2d'] += 1
                elif 'dense' in layer_name:
                    key = f"λd{counters['dense']}"
                    counters['dense'] += 1
                else:
                    key = f"λo{counters['other']}"
                    counters['other'] += 1
                
                # Format the lambda value
                lambda_info[key] = f"{pruning_layer.constraint_layer.lmbda.numpy():.2f}"
                
    return lambda_info


@tf.function
def train_step(model, inputs, logits, optimizer: keras.optimizers.Optimizer):
    with tf.GradientTape() as tape:
        outputs = model(inputs, training=True)
        loss = custom_loss(logits, outputs)
        loss += get_model_losses(model, losses=keras.ops.convert_to_tensor(0.))
        if model.losses:
            loss += tf.add_n(model.losses)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

@tf.function
def valid_step(model, inputs, logits):
    outputs = model(inputs, training=False)
    loss = custom_loss(logits, outputs)
    pquant_loss = get_model_losses(model, losses=keras.ops.convert_to_tensor(0.))
    if model.losses:
            loss += tf.add_n(model.losses)
    return loss + pquant_loss, pquant_loss

def train_smart_pixels_tf(model, epoch, trainloader, optimizer: keras.optimizers.Optimizer, *args, **kwargs):
    total_loss = 0.0
    pbar = tqdm(range(len(trainloader)), desc=f"Epoch {epoch} [Training]")
    
    for batch_idx in pbar:
        inputs, logits = trainloader[batch_idx]
        loss = train_step(model, inputs, logits, optimizer)
        total_loss += loss
        metrics = {
            'loss': f'{loss.numpy():.2f}',
        }
        metrics.update(get_lambda_values(model))
        pbar.set_postfix(**metrics)
        
    train_loader.on_epoch_end()
    return (total_loss/ len(trainloader)).numpy()

        

        
def validate_smart_pixels_tf(model, epoch, testloader, *args, **kwargs):
    total_loss = 0.0
    pbar = tqdm(range(len(testloader)), desc=f"Epoch {epoch} [Validation]")
    
    for batch_idx in pbar:
        inputs, logits = testloader[batch_idx]
        loss, pquant_loss = valid_step(model, inputs, logits)
        total_loss += loss
        pbar.set_postfix(loss=f'{loss.numpy():.2f}')

    testloader.on_epoch_end()
        
    avg_total_loss = total_loss / len(testloader)
    ratio = get_layer_keep_ratio(model)

    summary_string = (
        f"\tTotal Loss (NLL + MDMM): {avg_total_loss:.2f}\n"
        f"\tMDMM Loss:               {pquant_loss:.2f}\n"
        f"\tRemaining Weights:       {ratio * 100:.2f} %"
    )
    print(summary_string)
    return (total_loss/ len(testloader)).numpy()


## Create loss function, scheduler and optimizer

## Train model
Training time. We use the train_compressed_model function from pquant to train. We need to provide some parameters such as training and validation functions, their input parameters, the model and the config file. The function automatically adds pruning layers and replaces activations with a quantized variant, trains the model, and removes the pruning layers after training is done

In [25]:
for layer in model.layers:
        if "CompressedLayer" in layer.__class__.__name__ and hasattr(layer, 'pruning_layer'):
            pruning_layer = layer.pruning_layer # This is your MDMM instance
            
            # Check if the MDMM instance has the constraint_layer
            if hasattr(pruning_layer, 'constraint_layer'):
                constraint = pruning_layer.constraint_layer
                
                # Finally, check for 'lmbda' on the constraint layer
                if hasattr(constraint, 'lmbda'):
                    print(f"  Layer: {layer.name}, Lambda Value: {constraint.lmbda.numpy():.4f}")


  Layer: compressed_layer_conv2d_keras_3, Lambda Value: 0.0000
  Layer: compressed_layer_dense_keras_3, Lambda Value: 0.0000
  Layer: compressed_layer_dense_keras_4, Lambda Value: 0.0000
  Layer: compressed_layer_dense_keras_5, Lambda Value: 0.0000


In [26]:
from pathlib import Path
from secrets import token_hex

BASE_DIR = Path("/home/das214/PQuant/mdmm_dev/trainings").resolve()
RUN_DIR  = BASE_DIR / "smart_pixels"
FP       = token_hex(4)          

BASE_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIR.mkdir(parents=True, exist_ok=True)

RUN_OUT = RUN_DIR / FP
RUN_OUT.mkdir(exist_ok=True)

print(f"Training artifacts will be stored in: {RUN_OUT}")

Training artifacts will be stored in: /home/das214/PQuant/mdmm_dev/trainings/smart_pixels/e6697199


In [27]:
from pquant.core.tf_impl.callbacks import CSVLogger, EarlyStopping 
from pquant.core.tf_impl.callbacks import ModelCheckpoint, EpochCheckpoint

cbs = [
    CSVLogger(RUN_OUT / "history.csv"),                   
    EpochCheckpoint(RUN_OUT / "checkpoints"),              
    EarlyStopping(patience=50, min_delta=1e-3),            
]

In [ ]:
from pquant import iterative_train
"""
Inputs to train_resnet we defined previously are:
          model, trainloader, device, loss_func, epoch, optimizer, scheduler, **kwargs
"""

optimizer =  keras.optimizers.Nadam(learning_rate=config["lr"])

trained_model = iterative_train(model = model, 
                                config = config, 
                                train_func = train_smart_pixels_tf, 
                                valid_func = validate_smart_pixels_tf, 
                                trainloader = train_loader, 
                                testloader = val_loader, 
                                device = None, 
                                loss_func = custom_loss,
                                optimizer = optimizer, 
                                callbacks     = cbs, 
                                )

Training...


Epoch 0 [Training]:   0%|          | 0/84 [00:00<?, ?it/s]

Epoch 0 [Validation]: 100%|██████████| 21/21 [00:02<00:00,  7.45it/s, loss=16170.89]


	Total Loss (NLL + MDMM): 16036.61
	MDMM Loss:               4.96
	Remaining Weights:       99.61 %


Epoch 1 [Validation]: 100%|██████████| 21/21 [00:01<00:00, 15.17it/s, loss=8168.31]


...

Epoch 23 [Validation]: 100%|██████████| 21/21 [00:01<00:00, 14.95it/s, loss=-14593.58]


	Total Loss (NLL + MDMM): -14352.20
	MDMM Loss:               124.83
	Remaining Weights:       99.56 %


Epoch 24 [Validation]: 100%|██████████| 21/21 [00:01<00:00, 14.09it/s, loss=-15001.77]


	Total Loss (NLL + MDMM): -14760.04
	MDMM Loss:               130.97
	Remaining Weights:       99.72 %


Epoch 25 [Validation]:  48%|████▊     | 10/21 [00:00<00:00, 14.68it/s, loss=-15177.51]

In [ ]:
# TODO: 
#     - Increase number of epochs
#     - Implement early stopping
#     - Saving model weights (checkPoints)
#     - HLS4ml tutorial